In [1]:
#AutoEncoder 만들고
#Check 만들고
#Transfer learning

In [2]:
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
from torch import optim
from torch.utils.data import Dataset,DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

In [3]:
from sklearn.preprocessing import MinMaxScaler

In [4]:
raw=pd.read_csv('/nfs-data/OV/data/mother/TCGA_inter_SNUH_OV_FPKM_z_data.csv')

In [5]:
class Dataset(Dataset):
    def __init__(self,x):
        self.x=torch.from_numpy(x).cuda().float()
    def __len__(self):
        return self.x.size(0)
    def __getitem__(self,idx):
        return self.x[idx,:]

In [6]:
raw['label']=raw['patient'].apply(lambda x: 1 if x.__contains__('TCGA') else 0)

In [7]:
tcga,snuh=raw[raw['patient'].str.match('TCGA')],raw[~raw['patient'].str.match('TCGA')]

In [8]:
tcga_idx,snuh_idx=raw[raw['patient'].str.match('TCGA')]['patient'],raw[~raw['patient'].str.match('TCGA')]['patient']

In [9]:
tcga_scaler=MinMaxScaler()
snuh_scaler=MinMaxScaler()

In [10]:
tcga=tcga.drop(columns=['patient'])
snuh=snuh.drop(columns=['patient'])

In [11]:
tcga_columns=tcga.columns.values
snuh_columns=snuh.columns.values

In [12]:
nor_tcga=tcga_scaler.fit_transform(tcga)
nor_snuh=snuh_scaler.fit_transform(snuh)

In [13]:
nor_snuh[:,-1]=1

In [14]:
nor_stack=np.concatenate((nor_tcga,nor_snuh),axis=0)

In [15]:
x_tr,x_cv,_,_=train_test_split(nor_stack,nor_stack,test_size=0.1,stratify=nor_stack[:,-1])

In [16]:
tr_set=Dataset(x_tr[:,:-1])

In [17]:
cv_set=Dataset(x_cv[:,:-1])

In [18]:
Feature_num=x_tr[:,:-1].shape[1]

In [19]:
class Encoder(torch.nn.Module):
    def __init__(self):
        super(Encoder,self).__init__()
        self.encoder=nn.Sequential(
        nn.Linear(Feature_num,10000),
        nn.ReLU(True),
        nn.Linear(10000,5000),
        nn.ReLU(True),
        nn.Linear(5000,2500),
        nn.ReLU(True),
        nn.Linear(2500,1024),
        nn.ReLU(True),
        nn.Linear(1024,512),
        nn.ReLU(True),
        nn.Linear(512,256),
        nn.ReLU(True),
        nn.Linear(256,128))
    def forward(self,x):
        x=self.encoder(x)
        return x

In [20]:
class Decoder(torch.nn.Module):
    def __init__(self):
        super(Decoder,self).__init__()
        self.decoder=nn.Sequential(
        nn.Linear(128,256),
        nn.ReLU(True),
        nn.Linear(256,512),
        nn.ReLU(True),
        nn.Linear(512,1024),
        nn.ReLU(True),        
        nn.Linear(1024,2500),
        nn.ReLU(True),
        nn.Linear(2500,5000),
        nn.ReLU(True),
        nn.Linear(5000,10000),
        nn.ReLU(True),
        nn.Linear(10000,Feature_num))
        
    def forward(self,x):
        x=self.decoder(x)
        return x

In [21]:
import pytorch_lightning as pl

In [22]:
class AutoEncoder(pl.LightningModule):
    def __init__(self):
        super(AutoEncoder,self).__init__()
        self.encoder=Encoder()
        self.decoder=Decoder()
    def forward(self,x):
        return self.encoder(x)
    
    def training_step(self,batch,batch_idx):
        loss=self._shared_step(batch)
        #self.log('train_loss',loss)
        return {'loss':loss}
    
    def validation_step(self,batch,batch_idx):
        loss=self._shared_step(batch)
        #self.log('val_loss',loss)
        return {'loss':loss}
        
    def _shared_step(self,batch):
        x=batch
        #print(x)
        z=self.encoder(x)
        
        hat_x=self.decoder(z)
        reconstruction_loss=nn.functional.mse_loss(hat_x,x)
        return reconstruction_loss
    
    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(),lr=1e-05)

In [23]:
tr_loader=DataLoader(tr_set,batch_size=64)
cv_loader=DataLoader(cv_set)

In [24]:
autoencoder=AutoEncoder()

In [25]:
trainer=pl.Trainer(gpus=1)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
CUDA_VISIBLE_DEVICES: [0]


In [26]:
trainer.fit(autoencoder,tr_loader,cv_loader)

/home/learnmore/anaconda3/envs/pytorch1/lib/python3.6/site-packages/pytorch_lightning/utilities/distributed.py:24: RuntimeWarning: You have defined a `val_dataloader()` and have defined a `validation_step()`, you may also want to define `validation_epoch_end()` for accumulating stats.
  warnings.warn(*args, **kwargs)

  | Name    | Type    | Params
------------------------------------
0 | encoder | Encoder | 287 M 
1 | decoder | Decoder | 287 M 
/home/learnmore/anaconda3/envs/pytorch1/lib/python3.6/site-packages/pytorch_lightning/utilities/distributed.py:24: UserWarning: The dataloader, val dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 40 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)


/home/learnmore/anaconda3/envs/pytorch1/lib/python3.6/site-packages/pytorch_lightning/utilities/distributed.py:24: UserWarning: The dataloader, train dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 40 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)


/home/learnmore/anaconda3/envs/pytorch1/lib/python3.6/site-packages/pytorch_lightning/utilities/distributed.py:24: RuntimeWarning: The metric you returned None must be a `torch.Tensor` instance, checkpoint not saved HINT: what is the value of val_loss in validation_epoch_end()?
  warnings.warn(*args, **kwargs)
/home/learnmore/anaconda3/envs/pytorch1/lib/python3.6/site-packages/pytorch_lightning/utilities/distributed.py:24: RuntimeWarning: Can save best model only with val_loss available, skipping.
  warnings.warn(*args, **kwargs)


1

TypeError: 'TensorBoardLogger' object is not callable